In [1]:
from nfl import storage as nfl_db
from nfl.fdb import const as fdb_const
from nfl.fo import const as fo_const

# Team Mappings
Match FDB teams to FO teams and store result

In [11]:
fdb_teams = nfl_db.get_data(fdb_const.DB, fdb_const.COLL_TEAMS)
fdb_teams = fdb_teams[['Id', 'Name']].drop_duplicates()
fdb_teams

,Id,Name
0,miami-dolphins,Miami Dolphins
1,new-england-patriots,New England Patriots
2,new-york-jets,New York Jets
3,baltimore-ravens,Baltimore Ravens
4,cincinnati-bengals,Cincinnati Bengals
5,cleveland-browns,Cleveland Browns
6,pittsburgh-steelers,Pittsburgh Steelers
7,houston-texans,Houston Texans
8,indianapolis-colts,Indianapolis Colts
9,jacksonville-jaguars,Jacksonville Jaguars


In [20]:
# FO Notes on team names:
# Please note that "HOIL" refers to the Houston Oilers, "LARD" refers to the Los Angeles Raiders, 
# and "LARM" refers to the Los Angeles Rams. "CLE1" refers to the original Cleveland Browns and 
# choosing "CLE" in years prior to 1999 will give a blank result.
fo_teams = nfl_db.get_data(fo_const.DB, fo_const.COLL_TEAM)
fo_teams = list(fo_teams['TeamName']) + list(fo_teams['TeamNameAlt'].dropna())
fo_teams = np.unique(fo_teams)
fo_teams

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'CLE1',
       'DAL', 'DEN', 'DET', 'GB', 'HOIL', 'HOU', 'IND', 'JAC', 'KC',
       'LARD', 'LARM', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK',
       'PHI', 'PHX', 'PIT', 'SD', 'SEA', 'SF', 'STL', 'TB', 'TEN', 'WAS'], 
      dtype='<U4')

In [27]:
fdb_res = []
fdb_to_fo = {
    'new-england-patriots': 'NE',
    'new-york-jets': 'NYJ',
    'kansas-city-chiefs': 'KC',
    'san-diego-chargers': 'SD',
    'new-york-giants': 'NYG',
    'green-bay-packers': 'GB',
    'new-orleans-saints': 'NO',
    'tampa-bay-buccaneers': 'TB',
    'san-francisco-49ers': 'SF',
    'st-louis-rams': 'STL',
    'houston-texans': 'HOU',
    'houston-oilers': 'HOIL'
}
fo_to_fdb = {
    'CLE1': 'cleveland-browns',
    'HOIL': 'houston-oilers',
    'LARD': 'oakland-raiders',
    'LARM': 'st-louis-rams',
    'PHX': 'arizona-cardinals',
}
for i, r in fdb_teams.iterrows():
    fdb_id = r['Id']
    fdb_name = r['Name']
    if fdb_id in fdb_to_fo:
        fo_id = fdb_to_fo[fdb_id]
    else:
        fo_id = fdb_id[:3].upper()
    if fo_id in fo_teams:
        fdb_res.append((fdb_name, fdb_id, fo_id))
    else:
        print('No match found for FDB team {}'.format(fdb_id))
        
fdb_res = pd.DataFrame(fdb_res, columns=['TeamName', 'FDB_ID', 'FO_ID'])

fo_res = []
for fo_id in fo_teams:
    if fo_id not in fdb_res['FO_ID'].values:
        if fo_id not in fo_to_fdb:
            print('No match found for FO team {}'.format(fo_id))
        else:
            fdb_id = fo_to_fdb[fo_id]
            team_name = fdb_res[fdb_res['FDB_ID'] == fdb_id]['TeamName'].iloc[0]
            fo_res.append((team_name, fdb_id, fo_id))
fo_res = pd.DataFrame(fo_res, columns=fdb_res.columns)
res = fdb_res.append(fo_res)
res.groupby('TeamName').size().order()

TeamName
Kansas City Chiefs      1
Tennessee Oilers        1
Tampa Bay Buccaneers    1
Seattle Seahawks        1
San Francisco 49ers     1
San Diego Chargers      1
Pittsburgh Steelers     1
Philadelphia Eagles     1
New York Jets           1
New York Giants         1
New Orleans Saints      1
New England Patriots    1
Minnesota Vikings       1
Miami Dolphins          1
Tennessee Titans        1
Washington Redskins     1
Indianapolis Colts      1
Houston Texans          1
Houston Oilers          1
Green Bay Packers       1
Detroit Lions           1
Denver Broncos          1
Dallas Cowboys          1
Atlanta Falcons         1
Jacksonville Jaguars    1
Cincinnati Bengals      1
Chicago Bears           1
Carolina Panthers       1
Buffalo Bills           1
Baltimore Ravens        1
Cleveland Browns        2
Oakland Raiders         2
St. Louis Rams          2
Arizona Cardinals       2
dtype: int64

In [32]:
res[res['TeamName'] == 'Arizona Cardinals']

,TeamName,FDB_ID,FO_ID
27,Arizona Cardinals,arizona-cardinals,ARI
3,Arizona Cardinals,arizona-cardinals,PHX


In [34]:
from nfl.unified import const as unified_const
nfl_db.insert_data(unified_const.DB, unified_const.COLL_TEAM, res, ['FDB_ID', 'FO_ID'])